In [52]:
%%file py_scripts/generate_data.py

from __future__ import division
import numpy as np
import math
import time as time
import os

np.random.seed(1)

#weights matrix
A=np.array([[0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0]])

#data simulation
num_objects=100
object_dim=36
 
sigma_x_orig=0.5

I=sigma_x_orig*np.eye(object_dim)
Z_orig=np.zeros((num_objects,4))

X=np.zeros((num_objects,object_dim))

for i in range(num_objects):
    Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    while sum(Z_orig[i,:])==0:
        Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    X[i,:]=np.dot(np.random.normal(0,1,object_dim),I)+np.dot(Z_orig[i,:],A)

#creating folder if not present
if not os.path.exists('data_files'):
    os.makedirs('data_files')

#saving simulated data    
np.savetxt('data_files/Z_orig.csv', Z_orig, delimiter=',')
np.savetxt('data_files/X_orig.csv', X, delimiter=',')
np.savetxt('data_files/A_orig.csv', A, delimiter=',')

Writing py_scripts/generate_data.py


In [269]:
! python py_scripts/generate_data.py

In [53]:
%%file py_scripts/functions.py

from __future__ import division
import numpy as np
import math

np.random.seed(1)

#IBP function
def sampleIBP(alpha, num_objects):
    """generates the IBP prior"""
    result=np.zeros((num_objects, 1000))
    t=np.random.poisson(alpha)
    result[0,0:t]=np.ones(t)
    K_plus=t
    p=np.array((0,0))
    for i in range(2, num_objects+1):
        for j in range(K_plus):
            p[0]=np.log(sum(result[0:i,j]))-np.log(i) 
            p[1]=np.log(i - sum(result[0:i,j])) - np.log(i)
            p = np.exp(p-max(p))
            if np.random.uniform(0,1,1)<(p[0]/sum(p)):
                result[i-1,j]=1
            else:
                result[i-1,j]=0
        t=np.random.poisson(alpha/i)
        result[i-1,K_plus:K_plus+t]=np.ones(t)
        K_plus=K_plus+t
    
    result=result[:,0:K_plus]
    return(result,K_plus)     

#calcinverse proposed by Griffiths
def calcInverse(Zn,M,i,k,val):
    """calculates inverse based on Griffiths and Ghahramani approximation"""
    M_i = M-np.dot(np.dot(np.dot(M,Zn[i,:].T),Zn[i,:]),M)/(np.dot(np.dot(Zn[i,:],M),Zn[i,:].T)-1)
    Zn[i,k] = val
    M = M_i-np.dot(np.dot(np.dot(M_i,Zn[i,:].T),Zn[i,:]),M_i)/(np.dot(np.dot(Zn[i,:],M_i),Zn[i,:].T)+1)
    Inv = M
    return Inv

Writing py_scripts/functions.py


In [54]:
%%file py_scripts/likelihood_old.py

from __future__ import division
import numpy as np
import math

np.random.seed(1)

#old likelihood, slower
def likelihood_old(X, Z, M, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    """calculates the likelihood density value"""
    log_ll=(-1)*num_objects*object_dim*0.5*np.log(2*np.pi)-1*(num_objects-K_plus)*object_dim*np.log(sigma_X)-K_plus*object_dim*np.log(sigma_A)-object_dim*(0.5)*np.log(np.linalg.det((np.dot(Z.T,Z) + np.dot((sigma_X**2/sigma_A**2),np.eye(K_plus)))))+(-1/(2*sigma_X**2))*np.trace(np.dot(np.dot(X.T,(np.eye(num_objects)-np.dot(np.dot(Z,M),Z.T))),X))
    return log_ll

Writing py_scripts/likelihood_old.py


In [55]:
%%file py_scripts/likelihood.py

from __future__ import division
import numpy as np
import math

np.random.seed(1)

#new likelihood, removed redundancy
def likelihood(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    """calculates the likelihood density value, optimized version"""
    M1 = (np.dot(Z.T,Z) + np.dot((sigma_X**2/sigma_A**2),np.eye(K_plus)))
    log_ll=(-1)*num_objects*object_dim*0.5*np.log(2*np.pi)-1*(num_objects-K_plus)*object_dim*np.log(sigma_X)-K_plus*object_dim*np.log(sigma_A)-object_dim*(0.5)*np.log(np.linalg.det(M1))+(-1/(2*sigma_X**2))*np.trace(np.dot(np.dot(X.T,(np.eye(num_objects)-np.dot(np.dot(Z,np.linalg.inv(M1)),Z.T))),X))
    return log_ll

Writing py_scripts/likelihood.py


In [56]:
%%file py_scripts/sampler_old.py

from __future__ import division
import numpy as np
import math
from functions import sampleIBP
from likelihood_old import likelihood_old

np.random.seed(1)

#old sampler with redundancy
#gibbs and MH
def sampler_old(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects):
    """Gibbs and MH combined sampler"""
    HN=0.0
    for i in range(1,num_objects+1): 
        HN=HN+1.0/i

    K_inf=20

    sam=sampleIBP(alpha,num_objects)

    Z=sam[0]
    K_plus=sam[1]

    chain_Z=np.zeros((SAMPLE_SIZE,num_objects,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))

    s_counter=0
    for e in range(E):
        print e, K_plus, alpha
        if((e+1)>BURN_IN):
            chain_Z[s_counter,:,0:K_plus]=Z[:,0:K_plus]
            chain_K[s_counter]=K_plus
            chain_sigma_X[s_counter]=sigma_X
            chain_sigma_A[s_counter]=sigma_A
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1

        for i in range(num_objects):
            for k in range(K_plus):
                if (k+1)>K_plus: 
                    break
                if Z[i,k]>0:
                    if (sum(Z[:,k]) - Z[i,k])<=0:
                        Z[i,k]=0
                        Z[:,k:(K_plus-1)] = Z[:,(k+1):K_plus]
                        K_plus = K_plus-1
                        continue

                P=np.array([0,0])

                Z[i,k]=1
                M1=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
                P[0]=likelihood_old(X, Z[:,0:K_plus], M1, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(sum(Z[:,k])- Z[i,k]) -np.log(num_objects)

                Z[i,k]=0
                M2=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
                P[1]=likelihood_old(X, Z[:,0:K_plus], M2, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - sum(Z[:,k])) -np.log(num_objects)

                P=np.exp(P - max(P))

                if np.random.uniform(0,1,1)<(P[0]/(P[0]+P[1])):
                    Z[i,k] = 1
                    M = M1
                else:
                    Z[i,k] = 0
                    M = M2

            trun=np.zeros(5)
            alpha_N = alpha/num_objects

            for k_i in range(5):
                if Z.shape[1]>(K_plus+k_i):
                    Z[i,K_plus:(K_plus+k_i)]=1       
                else:
                    Ztemp=np.zeros((Z.shape[0],K_plus+k_i))
                    Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                    Ztemp[i,K_plus:(K_plus+k_i)] = 1
                    Z=Ztemp

                M=np.linalg.inv((np.dot(Z[:,0:(K_plus+k_i)].T,Z[:,0:(K_plus+k_i)]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye((K_plus+k_i)))))
                trun[k_i] = k_i*np.log(alpha_N) - alpha_N - np.log(math.factorial(k_i)) + likelihood_old(X, Z[:,0:(K_plus+k_i)], M, sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            Z[i,K_plus:K_plus+4] = 0
            trun = np.exp(trun - max(trun))
            trun = trun/sum(trun)
            p = np.random.uniform(0,1,1)
            t = 0
            
            #sample new dishes
            for k_i in range(5):
                t = t+trun[k_i]
                if p<t:
                    new_dishes = k_i
                    break
            if Z.shape[1]>(K_plus+new_dishes):
                Ztemp=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)]=1       
            else:
                Ztemp=np.zeros((Z.shape[0],K_plus+new_dishes))
                Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)] = 1

            Z=Ztemp
            K_plus = K_plus + new_dishes
        
        #Metropolis
        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus+new_dishes))))
        l_curr=likelihood_old(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
        if np.random.uniform(0,1,1)<.5:
            pr_sigma_X=sigma_X-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_X=sigma_X+np.random.uniform(0,1,1)/20

        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((pr_sigma_X[0])**2/(sigma_A)**2),np.eye(K_plus+new_dishes))))
        l_new_X=likelihood_old(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, pr_sigma_X[0], K_plus+new_dishes, num_objects, object_dim)
        acc_X=np.exp(min(0,l_new_X-l_curr))

        if np.random.uniform(0,1,1)<.5:
            pr_sigma_A=sigma_A-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_A=sigma_A+np.random.uniform(0,1,1)/20

        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((sigma_X)**2/(pr_sigma_A[0])**2),np.eye(K_plus+new_dishes))))
        l_new_A=likelihood_old(X, Z[:,0:(K_plus+new_dishes)], M, pr_sigma_A[0], sigma_X, K_plus+new_dishes, num_objects, object_dim)
        acc_A=np.exp(min(0,l_new_A-l_curr))

        if np.random.uniform(0,1,1)<acc_X:
            sigma_X=pr_sigma_X[0]

        if np.random.uniform(0,1,1)<acc_A:
            sigma_A=pr_sigma_A[0]

        alpha = np.random.gamma(1+K_plus, 1/(1+HN))
    
    return(chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z)

Writing py_scripts/sampler_old.py


In [57]:
%%file py_scripts/sampler.py

from __future__ import division
import numpy as np
import math
from functions import sampleIBP
from likelihood import likelihood

np.random.seed(1)

def sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects):
    """Gibbs and MH combined sampler, optimized"""
    HN=0.0
    for i in range(1,num_objects+1): 
        HN=HN+1.0/i

    K_inf=20

    sam=sampleIBP(alpha,num_objects)

    Z=sam[0]
    K_plus=sam[1]

    chain_Z=np.zeros((SAMPLE_SIZE,num_objects,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))
    
    P=np.array([0,0])
    
    s_counter=0
    for e in range(E):
        print e, K_plus, alpha
        if((e+1)>BURN_IN):
            chain_Z[s_counter,:,0:K_plus]=Z[:,0:K_plus]
            chain_K[s_counter]=K_plus
            chain_sigma_X[s_counter]=sigma_X
            chain_sigma_A[s_counter]=sigma_A
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1
        #gibbs
        for i in range(num_objects):
            for k in range(K_plus):
                if (k+1)>K_plus:
                    break
                if Z[i,k]>0:
                    if (sum(Z[:,k]) - Z[i,k])<=0:
                        Z[i,k]=0
                        Z[:,k:(K_plus-1)] = Z[:,(k+1):K_plus]
                        K_plus = K_plus-1
                        continue

                Z[i,k]=1
                P[0]=likelihood(X, Z[:,0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(sum(Z[:,k])- Z[i,k]) -np.log(num_objects)

                Z[i,k]=0
                P[1]=likelihood(X, Z[:,0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - sum(Z[:,k])) -np.log(num_objects)

                P=np.exp(P - max(P))

                if np.random.uniform(0,1,1)<(P[0]/(P[0]+P[1])):
                    Z[i,k] = 1
                else:
                    Z[i,k] = 0

            trun=np.zeros(5) 
            alpha_N = alpha/num_objects
            #new dishes
            for k_i in range(5):
                if Z.shape[1]>(K_plus+k_i):
                    Z[i,K_plus:(K_plus+k_i)]=1       
                else:
                    Ztemp=np.zeros((Z.shape[0],K_plus+k_i))
                    Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                    Ztemp[i,K_plus:(K_plus+k_i)] = 1
                    Z=Ztemp

                trun[k_i] = k_i*np.log(alpha_N) - alpha_N - np.log(math.factorial(k_i)) + likelihood(X, Z[:,0:(K_plus+k_i)], sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            Z[i,K_plus:K_plus+4] = 0
            trun = np.exp(trun - max(trun))
            trun = trun/sum(trun)
            p = np.random.uniform(0,1,1)
            t = 0
            for k_i in range(5):
                t = t+trun[k_i]
                if p<t:
                    new_dishes = k_i
                    break
            if Z.shape[1]>(K_plus+new_dishes):
                Ztemp=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)]=1       
            else:
                Ztemp=np.zeros((Z.shape[0],K_plus+new_dishes))
                Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)] = 1

            Z=Ztemp
            K_plus = K_plus + new_dishes
        #metropolis
        l_curr=likelihood(X, Z[:,0:(K_plus+new_dishes)], sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
        if np.random.uniform(0,1,1)<.5:
            pr_sigma_X=sigma_X-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_X=sigma_X+np.random.uniform(0,1,1)/20

        l_new_X=likelihood(X, Z[:,0:(K_plus+new_dishes)], sigma_A, pr_sigma_X[0], K_plus+new_dishes, num_objects, object_dim)
        acc_X=np.exp(min(0,l_new_X-l_curr))

        if np.random.uniform(0,1,1)<.5:
            pr_sigma_A=sigma_A-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_A=sigma_A+np.random.uniform(0,1,1)/20

        l_new_A=likelihood(X, Z[:,0:(K_plus+new_dishes)], pr_sigma_A[0], sigma_X, K_plus+new_dishes, num_objects, object_dim)
        acc_A=np.exp(min(0,l_new_A-l_curr))

        if np.random.uniform(0,1,1)<acc_X:
            sigma_X=pr_sigma_X[0]

        if np.random.uniform(0,1,1)<acc_A:
            sigma_A=pr_sigma_A[0]

        alpha = np.random.gamma(1+K_plus, 1/(1+HN))
    
    return(chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z)

Writing py_scripts/sampler.py


In [275]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [58]:
%%file py_scripts/cython_setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("cython_functions.pyx")
)

Writing py_scripts/cython_setup.py


In [59]:
%%file py_scripts/cython_functions.pyx

from __future__ import division
import numpy as np
import math

np.random.seed(1)
#cythonizing

def likelihood(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    """cythonized likelihood"""
    M1 = (np.dot(Z.T,Z) + np.dot((sigma_X**2/sigma_A**2),np.eye(K_plus)))
    log_ll=(-1)*num_objects*object_dim*0.5*np.log(2*np.pi)-1*(num_objects-K_plus)*object_dim*np.log(sigma_X)-K_plus*object_dim*np.log(sigma_A)-object_dim*(0.5)*np.log(np.linalg.det(M1))+(-1/(2*sigma_X**2))*np.trace(np.dot(np.dot(X.T,(np.eye(num_objects)-np.dot(np.dot(Z,np.linalg.inv(M1)),Z.T))),X))
    return log_ll

def sampleIBP(alpha, num_objects):
    """cythonized IBP"""
    result=np.zeros((num_objects, 1000))
    t=np.random.poisson(alpha)
    result[0,0:t]=np.ones(t)
    K_plus=t
    p=np.array((0,0))
    for i in range(2, num_objects+1):
        for j in range(K_plus):
            p[0]=np.log(sum(result[0:i,j]))-np.log(i)
            p[1]=np.log(i - sum(result[0:i,j])) - np.log(i)
            p = np.exp(p-max(p))
            if np.random.uniform(0,1,1)<(p[0]/sum(p)):
                result[i-1,j]=1
            else:
                result[i-1,j]=0
        t=np.random.poisson(alpha/i)
        result[i-1,K_plus:K_plus+t]=np.ones(t)
        K_plus=K_plus+t
    
    result=result[:,0:K_plus]
    return(result,K_plus)

Writing py_scripts/cython_functions.pyx


In [60]:
%%file py_scripts/sampler_cy.py

from __future__ import division
import numpy as np
import math
from cython_functions import sampleIBP, likelihood

def sampler_cy(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects):
    """cythonized sampler"""
    HN=0.0
    for i in range(1,num_objects+1): 
        HN=HN+1.0/i

    K_inf=20

    sam=sampleIBP(alpha,num_objects)

    Z=sam[0]
    K_plus=sam[1]

    chain_Z=np.zeros((SAMPLE_SIZE,num_objects,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))
    
    P=np.array([0,0])
    #gibbs
    s_counter=0
    for e in range(E):
        print e, K_plus, alpha
        if((e+1)>BURN_IN):
            chain_Z[s_counter,:,0:K_plus]=Z[:,0:K_plus]
            chain_K[s_counter]=K_plus
            chain_sigma_X[s_counter]=sigma_X
            chain_sigma_A[s_counter]=sigma_A
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1

        for i in range(num_objects):
            for k in range(K_plus):
                if (k+1)>K_plus:
                    break
                if Z[i,k]>0:
                    if (sum(Z[:,k]) - Z[i,k])<=0:
                        Z[i,k]=0
                        Z[:,k:(K_plus-1)] = Z[:,(k+1):K_plus]
                        K_plus = K_plus-1
                        continue

                Z[i,k]=1
                P[0]=likelihood(X, Z[:,0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(sum(Z[:,k])- Z[i,k]) -np.log(num_objects)

                Z[i,k]=0
                P[1]=likelihood(X, Z[:,0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - sum(Z[:,k])) -np.log(num_objects)

                P=np.exp(P - max(P))

                if np.random.uniform(0,1,1)<(P[0]/(P[0]+P[1])):
                    Z[i,k] = 1
                else:
                    Z[i,k] = 0

            trun=np.zeros(5) 
            alpha_N = alpha/num_objects

            for k_i in range(5):
                if Z.shape[1]>(K_plus+k_i):
                    Z[i,K_plus:(K_plus+k_i)]=1       
                else:
                    Ztemp=np.zeros((Z.shape[0],K_plus+k_i))
                    Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                    Ztemp[i,K_plus:(K_plus+k_i)] = 1
                    Z=Ztemp

                trun[k_i] = k_i*np.log(alpha_N) - alpha_N - np.log(math.factorial(k_i)) + likelihood(X, Z[:,0:(K_plus+k_i)], sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            Z[i,K_plus:K_plus+4] = 0
            trun = np.exp(trun - max(trun))
            trun = trun/sum(trun)
            p = np.random.uniform(0,1,1)
            t = 0
            
            #new dishes
            for k_i in range(5):
                t = t+trun[k_i]
                if p<t:
                    new_dishes = k_i
                    break
            if Z.shape[1]>(K_plus+new_dishes):
                Ztemp=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)]=1       
            else:
                Ztemp=np.zeros((Z.shape[0],K_plus+new_dishes))
                Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                Ztemp[i,K_plus:(K_plus+new_dishes)] = 1

            Z=Ztemp
            K_plus = K_plus + new_dishes
        #metropolis 
        l_curr=likelihood(X, Z[:,0:(K_plus+new_dishes)], sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
        if np.random.uniform(0,1,1)<.5:
            pr_sigma_X=sigma_X-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_X=sigma_X+np.random.uniform(0,1,1)/20

        l_new_X=likelihood(X, Z[:,0:(K_plus+new_dishes)], sigma_A, pr_sigma_X[0], K_plus+new_dishes, num_objects, object_dim)
        acc_X=np.exp(min(0,l_new_X-l_curr))

        if np.random.uniform(0,1,1)<.5:
            pr_sigma_A=sigma_A-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_A=sigma_A+np.random.uniform(0,1,1)/20

        l_new_A=likelihood(X, Z[:,0:(K_plus+new_dishes)], pr_sigma_A[0], sigma_X, K_plus+new_dishes, num_objects, object_dim)
        acc_A=np.exp(min(0,l_new_A-l_curr))

        if np.random.uniform(0,1,1)<acc_X:
            sigma_X=pr_sigma_X[0]

        if np.random.uniform(0,1,1)<acc_A:
            sigma_A=pr_sigma_A[0]

        alpha = np.random.gamma(1+K_plus, 1/(1+HN))
    
    return(chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z)

Writing py_scripts/sampler_cy.py


In [279]:
! python py_scripts/cython_setup.py build_ext --inplace

running build_ext


In [61]:
%%file py_scripts/main.py

from __future__ import division
import numpy as np
import math
import time as time
import pandas as pd
from sampler_old import sampler_old
from sampler import sampler
from sampler_cy import sampler_cy
import os

np.random.seed(1)
#saving data
X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")

#defining parameters
E=1000 #change to 1000
BURN_IN=0
SAMPLE_SIZE=E-BURN_IN
object_dim=X.shape[1]
num_objects=X.shape[0]

sigma_A=1.
sigma_X=1.
alpha=1.

np.random.seed(1)
#timing all 3 sampler versions
t0 = time.time()
chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z=sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)
t1 = time.time()
total_new=t1-t0

np.random.seed(1)

t0 = time.time()
chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z=sampler_old(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)
t1 = time.time()
total_old=t1-t0

np.random.seed(1)

t0 = time.time()
chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z=sampler_cy(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)
t1 = time.time()
total_cy=t1-t0

time=np.array((total_old,total_new,total_cy))
print time

#making tables
index=['Naive','Optimized','Cythonized']
columns=['Time (in secs)']

if not os.path.exists('data_files'):
    os.makedirs('data_files')
    
df = pd.DataFrame(np.hstack((total_old,total_new,total_cy)),index=index,columns=columns)
tab = df.to_latex()
text_file = open("data_files/times.tex", "w")
text_file.write(tab)
text_file.close()

Writing py_scripts/main.py


In [281]:
! python py_scripts/main.py

0 4 1.0
1 2 0.126674053442
2 1 0.0702021687993
3 1 0.477594379014
4 1 0.500702351241
5 1 0.8576840553
6 1 0.0488551635105
7 1 0.0406054546334
8 1 0.191544037014
9 1 0.423824093922
0 4 1.0
1 2 0.126674053442
2 1 0.0702021687993
3 1 0.477594379014
4 1 0.500702351241
5 1 0.8576840553
6 1 0.0488551635105
7 1 0.0406054546334
8 1 0.191544037014
9 1 0.423824093922
0 4 1.0
1 2 0.126674053442
2 1 0.0702021687993
3 1 0.477594379014
4 1 0.500702351241
5 1 0.8576840553
6 1 0.0488551635105
7 1 0.0406054546334
8 1 0.191544037014
9 1 0.423824093922
[ 1.79832196  1.50682592  1.46198702]


In [62]:
%%file py_scripts/likelihood_times.py

from __future__ import division
import numpy as np
import math
import time as time
import pandas as pd
import timeit
from likelihood_old import likelihood_old
from likelihood import likelihood
from functions import sampleIBP
import os

X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")
E=1000 #change to 1000
BURN_IN=0
SAMPLE_SIZE=E-BURN_IN
object_dim=X.shape[1]
num_objects=X.shape[0]

sigma_A=1.
sigma_X=1.
alpha=1.

sam=sampleIBP(alpha,num_objects)
Z=sam[0]
K_plus=sam[1]

M1=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
M2=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
M=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))

#timing the likelihoods
t0=time.time()
for i in range(1000):
    likelihood(X, Z[:,0:K_plus], sigma_A, sigma_X, K_plus, num_objects, object_dim)
t1=time.time()
elapsed1=t1-t0

t0=time.time()
for i in range(1000):
    M1=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
    likelihood_old(X, Z[:,0:K_plus], M1, sigma_A, sigma_X, K_plus, num_objects, object_dim)
t1=time.time()
elapsed2=t1-t0

elap=np.array((elapsed1,elapsed2))
print elap

#making table
index=['Old Likelihood','New Likelihood']
columns=['Time (in secs)']

if not os.path.exists('data_files'):
    os.makedirs('data_files')
    
df = pd.DataFrame(np.hstack((elapsed2,elapsed1)),index=index,columns=columns)
tab = df.to_latex()
text_file = open("data_files/times_like.tex", "w")
text_file.write(tab)
text_file.close()

Writing py_scripts/likelihood_times.py


In [511]:
! python py_scripts/likelihood_times.py

[ 0.14868498  0.27984309]


In [63]:
%%file py_scripts/compare_inverse.py
import time
import numpy as np
import pandas as pd
from functions import calcInverse, sampleIBP
import os
np.random.seed(1)

#load data
X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")

object_dim=X.shape[1]
num_objects=X.shape[0]

sigma_A=1.
sigma_X=1.
alpha=1.

i=12
k=3

#IBP prior
sam=sampleIBP(alpha,num_objects)
Z=sam[0]
K_plus=sam[1]
 
M = np.linalg.inv(Z.T.dot(Z)+(sigma_X**2/sigma_A**2)*np.identity(K_plus))
Z[i,k] = 1
val = 0

#timing matrix inversion
t0=time.time()
for l in range(1000):
    calcInverse(Z,M,i,k,val)
t1=time.time()
tcalc=t1-t0

t0=time.time()
for l in range(1000):
    np.linalg.inv(Z.T.dot(Z)+(sigma_X**2/sigma_A**2)*np.identity(K_plus))
t1=time.time()
tlinalg=t1-t0

print (tlinalg,tcalc)
columns = ['Time']
index = ['linalg.inv','calcInverse']
 
if not os.path.exists('data_files'):
    os.makedirs('data_files')
    
df = pd.DataFrame(np.hstack((tlinalg,tcalc)),columns=columns,index=index)
tab = df.to_latex()
text_file = open("data_files/inversetimes.tex", "w")
text_file.write(tab)
text_file.close()

Writing py_scripts/compare_inverse.py


In [504]:
! python py_scripts/compare_inverse.py

(0.024161815643310547, 0.016524076461791992)


In [64]:
%%file py_scripts/sampler_final.py

from __future__ import division
import numpy as np
import math
import time as time
from sampler import sampler
import os

X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")

E=1000
BURN_IN=0
SAMPLE_SIZE=E-BURN_IN
object_dim=X.shape[1]
num_objects=X.shape[0]

sigma_A=1.
sigma_X=1.
alpha=1.

#running the sampler and obtaining posteriors
np.random.seed(1)
chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z=sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)

if not os.path.exists('data_files'):
    os.makedirs('data_files')
    
np.save('data_files/chain_Z', chain_Z)
np.save('data_files/chain_K', chain_K)
np.save('data_files/chain_sigma_A', chain_sigma_A)
np.save('data_files/chain_sigma_X', chain_sigma_X)
np.save('data_files/chain_alpha', chain_alpha)
np.save('data_files/Z', Z)

Writing py_scripts/sampler_final.py


In [285]:
! python py_scripts/sampler_final.py

0 4 1.0
1 2 0.126674053442
2 1 0.0702021687993
3 1 0.477594379014
4 1 0.500702351241
5 1 0.8576840553
6 1 0.0488551635105
7 1 0.0406054546334
8 1 0.191544037014
9 1 0.423824093922


In [65]:
%%file py_scripts/figures.py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
plt.style.use('ggplot')
import os
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

#checking if folder exists
if not os.path.exists('data_files'):
    os.makedirs('data_files')

#obtaining histograms
chain_Z=np.load('data_files/chain_Z.npy')
chain_K=np.load('data_files/chain_K.npy')
chain_sigma_X=np.load('data_files/chain_sigma_X.npy')
chain_sigma_A=np.load('data_files/chain_sigma_A.npy')
chain_alpha=np.load('data_files/chain_alpha.npy')
Z=np.load('data_files/Z.npy')

plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(121)
plt.hist(chain_K[200:],bins=range(12),normed=True,color='black')
plt.xlabel('(a) Posterior distribution of K')
m=np.mean(chain_Z, axis=0)
plt.subplot(122)
plt.hist(np.ceil(np.sum(m,axis=1)),bins=range(10),normed=True,color='black')
plt.xlabel('(b) Features (ordered by frequency of occurence)')
plt.savefig('data_files/figures.png')

A=np.genfromtxt("data_files/A_orig.csv", delimiter=",")

#obtaining feature images
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(A[0,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(A[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(A[2,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(A[3,:].reshape(6,6),cmap=plt.cm.gray) 
plt.savefig('data_files/features.png')

Z=Z[:,0:4]
X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")

#obtaining data images
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(X[0,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(X[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(X[2,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(X[3,:].reshape(6,6),cmap=plt.cm.gray) 
plt.savefig('data_files/data.png')

A_inf=np.dot(np.dot(np.linalg.inv((np.dot(Z.T,Z)+(chain_sigma_X[999]**2/chain_sigma_A[999]**2)*np.eye(4))),Z.T),X)

#obtaining detected features images
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(A_inf[0,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(A_inf[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(A_inf[2,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(A_inf[3,:].reshape(6,6),cmap=plt.cm.gray)
plt.savefig('data_files/detected_features.png')

np.random.seed(1)

num_objects=100
object_dim=36
 
sigma_x_orig=0.5

I=sigma_x_orig*np.eye(object_dim)

Z_final=Z
X=np.zeros((num_objects,object_dim))

for i in range(num_objects):
    X[i,:]=np.dot(Z_final[i,:],A_inf[0:4,])

#obtaining reconstructed images
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(X[0,:].reshape(6,6),cmap=plt.cm.gray)
plt.subplot(142)
plt.pcolormesh(X[1,:].reshape(6,6),cmap=plt.cm.gray)
plt.subplot(143)
plt.pcolormesh(X[2,:].reshape(6,6),cmap=plt.cm.gray)
plt.subplot(144)
plt.pcolormesh(X[3,:].reshape(6,6),cmap=plt.cm.gray)
plt.savefig('data_files/detected_total_features.png')

index=['1st image','2nd image','3rd image','4th image']
columns=['F1','F2','F3','F4']

Z_orig=np.genfromtxt("data_files/Z_orig.csv", delimiter=",")
print Z_orig[0,:]
print Z_orig[1,:]
print Z_orig[2,:]
print Z_orig[3,:]

df = pd.DataFrame(np.hstack((np.vstack([Z_orig[0,0],Z_orig[1,0],Z_orig[2,0],Z_orig[3,0]]),np.vstack([Z_orig[0,1],Z_orig[1,1],Z_orig[2,1],Z_orig[3,1]]),np.vstack([Z_orig[0,2],Z_orig[1,2],Z_orig[2,2],Z_orig[3,2]]),np.vstack([Z_orig[0,3],Z_orig[1,3],Z_orig[2,3],Z_orig[3,3]]))),index=index,columns=columns)
tab = df.to_latex()
text_file = open("data_files/table_features.tex", "w")
text_file.write(tab)
text_file.close()

#making trace plots
plt.figure(num=None, figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(411)
plt.plot(chain_K,color='black')
plt.ylabel('K')
plt.subplot(412)
plt.plot(chain_alpha,color='black')
plt.ylabel('alpha')
plt.subplot(413)
plt.plot(chain_sigma_X,color='black')
plt.ylabel('sigmaX')
plt.subplot(414)
plt.plot(chain_sigma_A,color='black')
plt.ylabel('sigmaA')
plt.savefig('data_files/trace_plots.png')

Writing py_scripts/figures.py


In [365]:
! python py_scripts/figures.py

[ 0.  1.  0.  0.]
[ 1.  1.  0.  0.]
[ 1.  1.  0.  1.]
[ 1.  1.  0.  0.]


In [66]:
%%file py_scripts/unit_test.py

from __future__ import division
import numpy.testing as npt
import numpy as np
import math
import scipy.stats as stats
from functions import calcInverse, sampleIBP
from likelihood import likelihood

np.random.seed(1)

sigma_A=1.
sigma_X=1.
alpha=1.
num_objects=100
K_plus=4
alpha=1
object_dim=36

X=np.genfromtxt("data_files/X_orig.csv", delimiter=",")
chain_sigma_X=np.load('data_files/chain_sigma_X.npy')
chain_alpha=np.load('data_files/chain_alpha.npy')

Z=np.zeros((num_objects,100))

dish=0
while dish<num_objects:
    t=stats.poisson.rvs(alpha)
    if t>0:
        Z[dish,0:t]=1
        dish+=1

M=np.linalg.inv(np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus])+((sigma_X/sigma_A)**2)*np.identity(K_plus))

#UNIT TESTING

#calcInverse
def test1():
    inv=np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus])+((sigma_X/sigma_A)**2)*np.identity(K_plus)
    npt.assert_almost_equal(np.dot(M,inv),np.identity(K_plus))
    
Z = Z[:,0:K_plus]
Z_final=np.load('data_files/Z.npy')

#calcInverse
def test2():
    (i,k,val) = (7,1,1)
    M1=calcInverse(Z,M,i,k,val)
    Z[i,k]=val
    M_new=np.linalg.inv(np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus])+((sigma_X/sigma_A)**2)*np.identity(K_plus))
    np.allclose(M1,M_new)

#calcInverse
def test3():
    (i,k,val) = (9,3,0)
    M1=calcInverse(Z,M,i,k,val)
    Z[i,k]=val
    M_new=np.linalg.inv(np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus])+((sigma_X/sigma_A)**2)*np.identity(K_plus))
    np.allclose(M1,M_new)

#convergence of sampler for sigma X
def test4():
    assert (np.abs(np.mean(chain_sigma_X[200:])-0.5))<=.05

#convergence of sampler for alpha
def test5():
    assert (np.abs(np.mean(chain_alpha[200:])-1))<=.05

#testing likelihood to be > 0
def test6():
    like=np.exp(likelihood(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim))
    assert like>=0

#testing likelihood for -ve sigma A
def test7():
    assert(math.isnan(likelihood(X, Z, -0.5, sigma_X, K_plus, num_objects, object_dim)))

#testing likelihood for -ve sigma X
def test8():
    assert(math.isnan(likelihood(X, Z, sigma_A, -0.4, K_plus, num_objects, object_dim)))

#testing likelihood for div by 0
def test9():
    npt.assert_raises(ZeroDivisionError, likelihood, X, Z, 0, sigma_X, K_plus, num_objects, object_dim)

#testing that each object sampled atleast one feature
def test10():
    summ=np.sum(Z_final,axis=1)
    for i in range(100):
        assert summ[i] >=1
    

Writing py_scripts/unit_test.py


In [76]:
! py.test

============================= test session starts ==============================
platform darwin -- Python 2.7.9 -- py-1.4.25 -- pytest-2.6.3
collected 10 items 

py_scripts/unit_test.py ..........

========================== 10 passed in 0.71 seconds ===========================


In [ ]:
! pdflatex final_report

In [86]:
! make

pdflatex STA663_FinalReport_RadhikaAnand
This is pdfTeX, Version 3.14159265-2.6-1.40.15 (TeX Live 2014) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./STA663_FinalReport_RadhikaAnand.tex
LaTeX2e <2014/05/01>
Babel <3.9k> and hyphenation patterns for 78 languages loaded.
(/usr/local/texlive/2014/texmf-dist/tex/latex/base/article.cls
Document Class: article 2007/10/19 v1.4h Standard LaTeX document class
(/usr/local/texlive/2014/texmf-dist/tex/latex/base/size11.clo))
(/usr/local/texlive/2014/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/local/texlive/2014/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2014/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2014/texmf-dist/tex/generic/oberdiek/ifvtex.sty)
(/usr/local/texlive/2014/texmf-dist/tex/generic/ifxetex/ifxetex.sty))
(/usr/local/texlive/2014/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/local/texlive/2014/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/local/texl